## **04 Run NLP Model on Reviews**
This notebook takes the processed reviews from an earlier notebook, runs the spacy NLP model on the text to determine brewery offerings, and generates a csv file containing the data needed for the streamlit app.

### **Notebook Objectives**
1. Load the spacy NLP model previously trained following the project.yml file in the main directory
2. Apply the model to all reviews to determine mentions of brewering offerings
3. Group by brewery and sum all mentions of each brewery offering
4. Merge grouped dataframe with another that contains location information (city, address, lat/lon, etc.) for streamlit app purposes
5. Clean missing lat/lon info
6. Export final dataframe as csv for use by streamlit app

In [4]:
from pathlib import Path
import random
import math
import pandas as pd
import numpy as np
import json
from collections import Counter
random.seed(11)
import requests
import time
from dotenv import dotenv_values

import spacy

config = dotenv_values(dotenv_path=Path('../.env'))

In [5]:
# Import brewery review data into dataframe
filepath = Path('../assets/brewery_reviews.csv') 
reviews_df = pd.read_csv(filepath)
reviews_df.head()

,obdb_id,name,state,city,street,longitude,latitude,website_url,rating,review
0,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,"Tasty, fresh Brew. Went for a quick taste of t..."
1,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,Great local micro brewery. The beer here is ve...
2,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,Nice local SE MASS Brewery. Definitely worth t...
3,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,5.0,Great Beer. I come here often for some good be...
4,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,5.0,Micro Brewery with diverse collections . This ...


In [6]:
# Load labels from Doccano export
labels_filepath = Path('../configs/label_config.json')
labels = []
with open(labels_filepath) as f:
    labels_file = json.load(f)
    labels = [label['text'] for label in labels_file]
print(labels)

['VIBE', 'LOCATION', 'BEER', 'FEATURE', 'FOOD', 'GAMES', 'OUTDOOR', 'MUSIC', 'TOUR', 'DOG']


In [7]:
# Load spacy model
nlp = spacy.load(Path('../training/model-pattern/'))

# Extract entity matches in lists
def get_entities(nlp, labels, review):
    doc = nlp(review)
    entities = [ [] for _ in range(len(labels)) ]
    for ent in doc.ents:
        for index, label in enumerate(labels):
            if ent.label_ == label:
                entities[index].append(ent.text.lower())
    return entities

columns = [ [] for _ in range(len(labels)) ]
for review in reviews_df['review']:
    entities = get_entities(nlp, labels, review)
    counts = [Counter(x) for x in entities]
    for index, label in enumerate(counts):
        columns[index].append(label) 

In [8]:
# Inspect results
for index, label in enumerate(labels):
    reviews_df[label] = columns[index]
reviews_df.head()

,obdb_id,name,state,city,street,longitude,latitude,website_url,rating,review,VIBE,LOCATION,BEER,FEATURE,FOOD,GAMES,OUTDOOR,MUSIC,TOUR,DOG
0,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,"Tasty, fresh Brew. Went for a quick taste of t...",{'friendly': 1},{},"{'pale ale': 1, 'porter': 1}",{},"{'food truck': 1, 'lunch': 2, 'food': 1}",{},{},{},{},{}
1,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,Great local micro brewery. The beer here is ve...,{'friendly': 1},{},{'pale ale': 1},{},{},{},{},{},{},{}
2,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,4.0,Nice local SE MASS Brewery. Definitely worth t...,{},{},{},{},{'food': 1},{},{},{},{},{}
3,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,5.0,Great Beer. I come here often for some good be...,{'friendly': 1},{},{},{},{},{},{},{},{},{}
4,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,5.0,Micro Brewery with diverse collections . This ...,{'friendly': 1},{},{'pale ale': 1},{},{},{},{},{},{},{}


In [17]:
# Inspect review and feature matches
index = random.randint(0, len(reviews_df))
print(f'Index: {index}')
print(reviews_df['review'][index])
for label in labels:
    print(reviews_df[label][index])

Index: 2096
Great Brewery. I usually stop there on Fridays on my way home for a growler or a few cans of beer. I have gone there with friends and grabbed a table for a few. We have gotten take out food a  local restaurants and eaten there while we drink beer. They have good flights of beer. I highly recommend this place to stop and have a beer or two.
Counter()
Counter()
Counter({'growler': 1})
Counter()
Counter({'food': 1})
Counter()
Counter()
Counter()
Counter()
Counter()


In [24]:
# Group by brewery
aggregate = {}
columns = ['name', 'state', 'city', 'street', 'longitude', 'latitude', 'website_url']
for column in columns:
    aggregate[column] = 'max'
aggregate['review'] = 'count'
# add labels to agg function
for label in labels:
    aggregate[label] = 'sum'
brewery_features = reviews_df.groupby(by='obdb_id').agg(aggregate).reset_index()
print(brewery_features.shape)
# brewery_features.tail(10)

(113, 19)


In [25]:
# Merge any previously computed lat lon corrections
filepath = Path('../assets/breweries_app.csv')
brewery_lat_lon = pd.read_csv(filepath)[['name', 'longitude', 'latitude']]
# brewery_lat_lon.head()
brewery_features = brewery_features.merge(brewery_lat_lon, on='name', how='left', suffixes=(None,'_x'))
print(f'Shape: {brewery_features.shape}')
brewery_features.tail(5)

Shape: (113, 21)


,obdb_id,name,state,city,street,longitude,latitude,website_url,review,VIBE,...,BEER,FEATURE,FOOD,GAMES,OUTDOOR,MUSIC,TOUR,DOG,longitude_x,latitude_x
108,wandering-star-brewing-co-pittsfield,Wandering Star Brewing Co,Massachusetts,Pittsfield,11 Gifford St,-73.212023,42.457064,http://www.wanderingstarbrewing.com,13,"{'friendly': 7, 'fun': 3}",...,"{'growlers': 6, 'porter': 3, 'lager': 1, 'stou...",{'tv': 1},"{'snacks': 1, 'food': 2}","{'pool': 1, 'games': 2}",{'outdoor': 1},{'music': 1},{},"{'dog': 3, 'dogs': 1}",-73.212023,42.457064
109,westfield-river-brewing-co-southwick,Westfield River Brewing Co,Massachusetts,Southwick,707 College Hwy,-72.765248,42.062268,http://www.westfieldriverbrewing.com,30,"{'cool': 4, 'friendly': 4, 'amazing': 1, 'fun'...",...,"{'ipa': 1, 'ipas': 1, 'stout': 1}",{'tvs': 1},"{'food': 24, 'pizzas': 2, 'chicken': 6, 'pizza...",{},"{'outdoor': 1, 'patio': 1, 'outside': 2}","{'music': 3, 'bands': 1, 'band': 1}",{},{},-72.765248,42.062268
110,widowmaker-brewing-braintree,Widowmaker Brewing,Massachusetts,Braintree,220 Wood Rd,NaN,NaN,http://www.widowmakerbrew.wordpress.com,21,"{'friendly': 11, 'fun': 3, 'cool': 1, 'comfort...",...,"{'ipas': 8, 'ipa': 2, 'stout': 4, 'stouts': 1}","{'tv': 3, 'tvs': 1}","{'food truck': 2, 'snacks': 4, 'food': 6, 'san...","{'games': 12, 'corn hole': 1}","{'outside': 2, 'outdoor': 3}",{'music': 2},{},"{'dog': 4, 'dogs': 3}",-71.036946,42.220592
111,winter-hill-brewing-company-somerville,Winter Hill Brewing Company,Massachusetts,Somerville,328 Broadway,NaN,NaN,http://www.winterhillbrewing.com,38,"{'comfortable': 1, 'friendly': 3, 'cool': 2, '...",...,"{'ipa': 8, 'pilsner': 4, 'pale ale': 4, 'porte...",{},"{'food': 36, 'dinner': 1, 'sandwich': 5, 'sand...",{},"{'backyard': 2, 'patio': 5, 'outdoor': 2}",{},{},{'dog': 2},-71.093978,42.392656
112,wormtown-brewery-worcester,Wormtown Brewery,Massachusetts,Worcester,72 Shrewsbury St,-71.791226,42.263499,http://www.wormtownbrewery.com,29,"{'fun': 6, 'friendly': 4, 'cool': 5, 'amazing'...",...,"{'growler': 1, 'pilsner': 1, 'ipa': 3, 'stout'...",{},"{'dinner': 1, 'pizza': 3, 'food': 3}",{'games': 1},"{'outside': 4, 'outdoor': 3, 'patio': 2}",{'music': 3},{},{},-71.791226,42.263499


In [26]:
# Remove any entry with unknown address
for index, row in brewery_features.iterrows():
    if pd.isnull(row['latitude']):
        brewery_features.at[index, 'latitude'] = brewery_features.at[index, 'latitude_x']
        brewery_features.at[index, 'longitude'] = brewery_features.at[index, 'longitude_x']
brewery_features = brewery_features.drop(columns = ['latitude_x', 'longitude_x'])
brewery_features.tail(5)

,obdb_id,name,state,city,street,longitude,latitude,website_url,review,VIBE,LOCATION,BEER,FEATURE,FOOD,GAMES,OUTDOOR,MUSIC,TOUR,DOG
108,wandering-star-brewing-co-pittsfield,Wandering Star Brewing Co,Massachusetts,Pittsfield,11 Gifford St,-73.212023,42.457064,http://www.wanderingstarbrewing.com,13,"{'friendly': 7, 'fun': 3}",{'warehouse': 1},"{'growlers': 6, 'porter': 3, 'lager': 1, 'stou...",{'tv': 1},"{'snacks': 1, 'food': 2}","{'pool': 1, 'games': 2}",{'outdoor': 1},{'music': 1},{},"{'dog': 3, 'dogs': 1}"
109,westfield-river-brewing-co-southwick,Westfield River Brewing Co,Massachusetts,Southwick,707 College Hwy,-72.765248,42.062268,http://www.westfieldriverbrewing.com,30,"{'cool': 4, 'friendly': 4, 'amazing': 1, 'fun'...",{},"{'ipa': 1, 'ipas': 1, 'stout': 1}",{'tvs': 1},"{'food': 24, 'pizzas': 2, 'chicken': 6, 'pizza...",{},"{'outdoor': 1, 'patio': 1, 'outside': 2}","{'music': 3, 'bands': 1, 'band': 1}",{},{}
110,widowmaker-brewing-braintree,Widowmaker Brewing,Massachusetts,Braintree,220 Wood Rd,-71.036946,42.220592,http://www.widowmakerbrew.wordpress.com,21,"{'friendly': 11, 'fun': 3, 'cool': 1, 'comfort...",{'spacious': 3},"{'ipas': 8, 'ipa': 2, 'stout': 4, 'stouts': 1}","{'tv': 3, 'tvs': 1}","{'food truck': 2, 'snacks': 4, 'food': 6, 'san...","{'games': 12, 'corn hole': 1}","{'outside': 2, 'outdoor': 3}",{'music': 2},{},"{'dog': 4, 'dogs': 3}"
111,winter-hill-brewing-company-somerville,Winter Hill Brewing Company,Massachusetts,Somerville,328 Broadway,-71.093978,42.392656,http://www.winterhillbrewing.com,38,"{'comfortable': 1, 'friendly': 3, 'cool': 2, '...",{},"{'ipa': 8, 'pilsner': 4, 'pale ale': 4, 'porte...",{},"{'food': 36, 'dinner': 1, 'sandwich': 5, 'sand...",{},"{'backyard': 2, 'patio': 5, 'outdoor': 2}",{},{},{'dog': 2}
112,wormtown-brewery-worcester,Wormtown Brewery,Massachusetts,Worcester,72 Shrewsbury St,-71.791226,42.263499,http://www.wormtownbrewery.com,29,"{'fun': 6, 'friendly': 4, 'cool': 5, 'amazing'...",{'warehouse': 1},"{'growler': 1, 'pilsner': 1, 'ipa': 3, 'stout'...",{},"{'dinner': 1, 'pizza': 3, 'food': 3}",{'games': 1},"{'outside': 4, 'outdoor': 3, 'patio': 2}",{'music': 3},{},{}


In [27]:
# Remove any entry with unknown address
for index, row in brewery_features.iterrows():
    if pd.isnull(row['street']):
        print(f"Removing {row['name']} with missing address.")
        brewery_features = brewery_features.drop(row.name)
print(brewery_features.shape)
brewery_features.head(5)

Removing Backlash Beer Co with missing address.
(112, 19)


,obdb_id,name,state,city,street,longitude,latitude,website_url,review,VIBE,LOCATION,BEER,FEATURE,FOOD,GAMES,OUTDOOR,MUSIC,TOUR,DOG
0,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,7,{'friendly': 4},{},"{'pale ale': 3, 'porter': 1, 'growler': 2, 'st...",{},"{'food truck': 2, 'lunch': 2, 'food': 2}",{},{},{},{},{}
1,3cross-fermentation-cooperative-worcester,3cross Fermentation Cooperative,Massachusetts,Worcester,4 Knowlton Ave,-71.830576,42.243649,http://www.3cross.coop,12,"{'cool': 2, 'friendly': 3, 'fun': 2, 'comforta...",{},"{'pale ale': 1, 'growler': 1, 'growlers': 1, '...",{'tv': 1},{'food': 6},{},{},"{'bands': 1, 'music': 2}",{},{}
2,7th-wave-brewing-medfield,7th Wave Brewing,Massachusetts,Medfield,120 N Meadows Rd,-71.328277,42.193576,http://www.7thwavebrewing.com,10,"{'amazing': 3, 'cool': 2, 'friendly': 3, 'fun'...",{},"{'stout': 1, 'ipa': 2, 'pale ale': 1}",{},"{'pizza': 14, 'pizzas': 2}",{'games': 2},{'outdoor': 1},{},{},{}
3,abandoned-building-brewery-easthampton,Abandoned Building Brewery,Massachusetts,Easthampton,142 Pleasant St Unit,NaN,NaN,http://www.abandonedbuildingbrewery.com,43,"{'fun': 6, 'friendly': 11, 'comfortable': 3, '...","{'old mill': 1, 'mill': 1}","{'ipa': 4, 'ipas': 2, 'stout': 4, 'growlers': 1}",{},"{'food': 6, 'snacks': 2}",{'games': 8},{},"{'band': 2, 'music': 6}",{'tour': 1},{'dog': 4}
4,aeronaut-brewing-company-somerville,Aeronaut Brewing Company,Massachusetts,Somerville,14 Tyler St,-71.106268,42.381972,http://www.aeronautbrewing.com,30,"{'friendly': 5, 'relaxed': 2, 'fun': 14, 'cool...",{'warehouse': 3},"{'lager': 2, 'growler': 1, 'ipa': 2, 'ipaa': 1...",{},"{'dinner': 2, 'food': 4, 'pizza': 1}",{'games': 15},{'outside': 1},"{'music': 7, 'band': 1}","{'touristy': 1, 'tour': 1}",{}


In [88]:
# Clean missing lat/lon data
def geocode(address):
    params = { 'format': 'json', 
               'addressdetails': 1, 
               'q': address}
    headers = { 'user-agent'   : config['USER_AGENT'] } 
    return requests.get('http://nominatim.openstreetmap.org/search', 
                        params=params, headers=headers)

count = 0
for index, row in brewery_features.iterrows():
    if pd.isnull(row['longitude']):
        print(f"Searching address for {row['name']}...")
        address = f"{row['street']}, {row['city']} {row['state']}"
        response = geocode(f"{row['street']}, {row['city']} {row['state']}")
        if response.json():
            print(f'Lat lon found!')
            count += 1
            brewery_features.at[index, 'latitude'] = response.json()[0]['lat']
            brewery_features.at[index, 'longitude'] = response.json()[0]['lon']
        time.sleep(2)
print(f'Added lat lon info for {count} entries.')

Searching address for Abandoned Building Brewery...
Searching address for Altruist Brewing Company...
Searching address for Black Hat Brew Works...
Searching address for Bright Ideas Brewing...
Searching address for Hog Island Beer Company...
Searching address for Shovel Town Brewery, Inc....
Added lat lon info for 0 entries.


In [28]:
# Inspect null fraction
print(brewery_features.shape)
print(f'Fraction of MA breweries containing nulls:\n{brewery_features.isnull().mean()}')

(112, 19)
Fraction of MA breweries containing nulls:
obdb_id        0.000000
name           0.000000
state          0.000000
city           0.000000
street         0.000000
longitude      0.053571
latitude       0.053571
website_url    0.080357
review         0.000000
VIBE           0.000000
LOCATION       0.000000
BEER           0.000000
FEATURE        0.000000
FOOD           0.000000
GAMES          0.000000
OUTDOOR        0.000000
MUSIC          0.000000
TOUR           0.000000
DOG            0.000000
dtype: float64


In [29]:
# Drop remaining entries with unknown lat/lon
brewery_features = brewery_features.dropna(subset=['latitude'])
print(brewery_features.shape)

(106, 19)


In [30]:
# Add columns for app filters
filt_columns = ['outdoor_', 'food_', 'dog_', 'games_', 'music_', 'tour_']
for col in filt_columns:
    # brewery_features[col] = np.zeros(brewery_features.shape[0], dtype=int)
    brewery_features[col] = ['No'] * brewery_features.shape[0]
brewery_features.head()

,obdb_id,name,state,city,street,longitude,latitude,website_url,review,VIBE,...,OUTDOOR,MUSIC,TOUR,DOG,outdoor_,food_,dog_,games_,music_,tour_
0,10th-district-brewing-company-abington,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,7,{'friendly': 4},...,{},{},{},{},No,No,No,No,No,No
1,3cross-fermentation-cooperative-worcester,3cross Fermentation Cooperative,Massachusetts,Worcester,4 Knowlton Ave,-71.830576,42.243649,http://www.3cross.coop,12,"{'cool': 2, 'friendly': 3, 'fun': 2, 'comforta...",...,{},"{'bands': 1, 'music': 2}",{},{},No,No,No,No,No,No
2,7th-wave-brewing-medfield,7th Wave Brewing,Massachusetts,Medfield,120 N Meadows Rd,-71.328277,42.193576,http://www.7thwavebrewing.com,10,"{'amazing': 3, 'cool': 2, 'friendly': 3, 'fun'...",...,{'outdoor': 1},{},{},{},No,No,No,No,No,No
4,aeronaut-brewing-company-somerville,Aeronaut Brewing Company,Massachusetts,Somerville,14 Tyler St,-71.106268,42.381972,http://www.aeronautbrewing.com,30,"{'friendly': 5, 'relaxed': 2, 'fun': 14, 'cool...",...,{'outside': 1},"{'music': 7, 'band': 1}","{'touristy': 1, 'tour': 1}",{},No,No,No,No,No,No
6,amherst-brewing-co-hangar-pub-and-grill-amherst,Amherst Brewing Co / Hangar Pub and Grill,Massachusetts,Amherst,10 University Dr,-72.531842,42.372864,http://www.amherstbrewing.com,30,"{'fun': 8, 'friendly': 5, 'accommodating': 1, ...",...,{},{'music': 1},{},{},No,No,No,No,No,No


In [33]:
# Inspect individual brewery
index = random.randint(0, brewery_features.shape[0])
brewery_features.loc[index,'VIBE']

Counter({'fun': 2, 'friendly': 9, 'cool': 8, 'amazing': 1, 'hidden gem': 1})

In [34]:
# Assign filters values
label_columns = [label.strip('_').upper() for label in filt_columns]
threshold = 4 # min number of reviews to have confidence
for index, row in brewery_features.iterrows():
    for label, filter in zip(label_columns, filt_columns):
        if row[label] and row['review'] >= threshold:
            brewery_features.at[index, filter] = 'Yes'
        elif row['review'] < threshold:
            brewery_features.at[index, filter] = 'Unknown'

In [36]:
# Inspect
brewery_features.tail(5)

,obdb_id,name,state,city,street,longitude,latitude,website_url,review,VIBE,...,OUTDOOR,MUSIC,TOUR,DOG,outdoor_,food_,dog_,games_,music_,tour_
108,wandering-star-brewing-co-pittsfield,Wandering Star Brewing Co,Massachusetts,Pittsfield,11 Gifford St,-73.212023,42.457064,http://www.wanderingstarbrewing.com,13,"{'friendly': 7, 'fun': 3}",...,{'outdoor': 1},{'music': 1},{},"{'dog': 3, 'dogs': 1}",Yes,Yes,Yes,Yes,Yes,No
109,westfield-river-brewing-co-southwick,Westfield River Brewing Co,Massachusetts,Southwick,707 College Hwy,-72.765248,42.062268,http://www.westfieldriverbrewing.com,30,"{'cool': 4, 'friendly': 4, 'amazing': 1, 'fun'...",...,"{'outdoor': 1, 'patio': 1, 'outside': 2}","{'music': 3, 'bands': 1, 'band': 1}",{},{},Yes,Yes,No,No,Yes,No
110,widowmaker-brewing-braintree,Widowmaker Brewing,Massachusetts,Braintree,220 Wood Rd,-71.036946,42.220592,http://www.widowmakerbrew.wordpress.com,21,"{'friendly': 11, 'fun': 3, 'cool': 1, 'comfort...",...,"{'outside': 2, 'outdoor': 3}",{'music': 2},{},"{'dog': 4, 'dogs': 3}",Yes,Yes,Yes,Yes,Yes,No
111,winter-hill-brewing-company-somerville,Winter Hill Brewing Company,Massachusetts,Somerville,328 Broadway,-71.093978,42.392656,http://www.winterhillbrewing.com,38,"{'comfortable': 1, 'friendly': 3, 'cool': 2, '...",...,"{'backyard': 2, 'patio': 5, 'outdoor': 2}",{},{},{'dog': 2},Yes,Yes,Yes,No,No,No
112,wormtown-brewery-worcester,Wormtown Brewery,Massachusetts,Worcester,72 Shrewsbury St,-71.791226,42.263499,http://www.wormtownbrewery.com,29,"{'fun': 6, 'friendly': 4, 'cool': 5, 'amazing'...",...,"{'outside': 4, 'outdoor': 3, 'patio': 2}",{'music': 3},{},{},Yes,Yes,No,Yes,Yes,No


In [37]:
# Generate lighter dataframe for app deployment
breweries_app = brewery_features.drop(['obdb_id', 'review'], axis=1).drop(labels, axis=1)
breweries_app.head()

,name,state,city,street,longitude,latitude,website_url,outdoor_,food_,dog_,games_,music_,tour_
0,10th District Brewing Company,Massachusetts,Abington,491 Washington St,-70.945941,42.105918,http://www.10thdistrictbrewing.com,No,Yes,No,No,No,No
1,3cross Fermentation Cooperative,Massachusetts,Worcester,4 Knowlton Ave,-71.830576,42.243649,http://www.3cross.coop,No,Yes,No,No,Yes,No
2,7th Wave Brewing,Massachusetts,Medfield,120 N Meadows Rd,-71.328277,42.193576,http://www.7thwavebrewing.com,Yes,Yes,No,Yes,No,No
4,Aeronaut Brewing Company,Massachusetts,Somerville,14 Tyler St,-71.106268,42.381972,http://www.aeronautbrewing.com,Yes,Yes,No,Yes,Yes,Yes
6,Amherst Brewing Co / Hangar Pub and Grill,Massachusetts,Amherst,10 University Dr,-72.531842,42.372864,http://www.amherstbrewing.com,No,Yes,No,Yes,Yes,No


In [38]:
# Inspect examples with 'Unknown' filter values
breweries_app.query("food_ == 'Unknown'")

,name,state,city,street,longitude,latitude,website_url,outdoor_,food_,dog_,games_,music_,tour_
8,Anawan Brewing Company,Massachusetts,Dighton,7 Park St,-71.128681,41.865120,http://www.anawanbrewingco.com,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
26,Building 8 Brewing,Massachusetts,Florence,320 Riverside Dr,-72.665272,42.320271,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
51,Hitchcock Brewing Company,Massachusetts,Whately,129 Christian Ln,-72.606412,42.446065,http://www.hitchcockbrewing.com,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
60,Kismet Brewing Company,Massachusetts,Westfield,66 S Broad St,-72.746109,42.110276,http://www.kismetbrewing.com,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
88,Second Wind Brewing Company,Massachusetts,Plymouth,7 Howland St,-70.666548,41.958555,http://www.secondwindbrewing.com,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
92,Spencer Brewery,Massachusetts,Spencer,167 N Spencer Rd,-72.013316,42.297633,http://www.spencerbrewery.com,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [124]:
# Export dataframe for app
filepath = Path('../assets/breweries_app.csv') 
breweries_app.to_csv(filepath, index=False)